<a href="https://colab.research.google.com/github/UNT-CSCE-Projects/US_stock_market_prediction/blob/ml-farhan/Optiver_Trading_at_the_close.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"farhanarrafi","key":"9bd5ba3bda1b77f0af3eddc2a8ff528d"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions list

In [7]:
! kaggle competitions download -c 'optiver-trading-at-the-close'

 93% 186M/201M [00:00<00:00, 246MB/s]
100% 201M/201M [00:00<00:00, 233MB/s]


In [8]:
! mkdir train

In [9]:
! unzip optiver-trading-at-the-close.zip -d train

Archive:  optiver-trading-at-the-close.zip
  inflating: train/example_test_files/revealed_targets.csv  
  inflating: train/example_test_files/sample_submission.csv  
  inflating: train/example_test_files/test.csv  
  inflating: train/optiver2023/__init__.py  
  inflating: train/optiver2023/competition.cpython-310-x86_64-linux-gnu.so  
  inflating: train/public_timeseries_testing_util.py  
  inflating: train/train.csv         
